# PlateNet AI - License Plate Detection Training
## YOLOv8 Fine-tuning for License Plate Recognition

### Project Overview
This notebook implements a complete license plate detection system using YOLOv8 (You Only Look Once version 8) deep learning model. The system is designed to detect and localize license plates in images and videos with high accuracy.

### Key Features
- **Transfer Learning**: Uses pre-trained YOLOv8 nano model for faster convergence
- **Google Colab Integration**: Optimized for Colab GPU environment
- **Automated Pipeline**: Complete workflow from dataset upload to model deployment
- **Professional Documentation**: Comprehensive comments and explanations

### Dataset Requirements
The training dataset should be in YOLO format containing:
- **Images**: License plate images in JPG/PNG format
- **Labels**: Corresponding annotation files in YOLO format (.txt)
- **Configuration**: YAML file defining dataset structure and classes

### Training Configuration
- **Model**: YOLOv8 Nano (yolov8n.pt)
- **Input Size**: 640x640 pixels
- **Epochs**: 50 (with early stopping)
- **Batch Size**: 16
- **Optimizer**: AdamW
- **Device**: GPU (Tesla T4 in Colab)

### Output
- **Trained Model**: best.pt (best performing weights)
- **Training Logs**: Complete training metrics and visualizations
- **Model Storage**: Automatic backup to Google Drive

---
**Author**: [Your Name]  
**Date**: [Current Date]  
**Version**: 1.0.0


In [ ]:
# Check GPU availability and status
# This command displays NVIDIA GPU information including:
# - GPU model and driver version
# - CUDA version compatibility
# - Memory usage and utilization
# - Temperature and power consumption
!nvidia-smi

Mon Aug 11 09:08:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Dataset Upload and Extraction
# =============================
# This cell handles the upload and extraction of the training dataset
# The dataset should be a ZIP file containing:
# - yolov8n.pt: Pre-trained YOLOv8 nano model weights
# - datasets/plates_yolo/: YOLO-formatted dataset with images and labels

from google.colab import files

# Upload the dataset ZIP file from local machine
uploaded = files.upload()              # Select your dataset.zip file
zip_name = next(iter(uploaded))        # Get the name of the uploaded file

# Create dataset directory and extract the ZIP file
!mkdir -p /content/dataset
!unzip -q -o "$zip_name" -d /content/dataset

Saving plates_yolo.zip to plates_yolo (1).zip


In [ ]:
# Install Required Dependencies
# =============================
# Install the necessary Python packages for YOLO training and computer vision
# - ultralytics: YOLOv8 implementation and training framework
# - opencv-python: Computer vision library for image processing

!pip install -q ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.3 MB/s eta 0:00:00


In [ ]:
# Dataset Structure Verification
# =============================
# Verify that the dataset files are properly extracted and located
# This step ensures all required files are in the correct locations before training

import glob, os

# Search for YOLO configuration file (yolo.yaml)
print("YOLO configuration files found:")
print(glob.glob("/content/dataset/**/yolo.yaml", recursive=True))

# Search for pre-trained model weights (yolov8n.pt)
print("\nPre-trained model weights found:")
print(glob.glob("/content/dataset/**/yolov8n.pt", recursive=True))

['/content/dataset/plates_yolo/yolo.yaml']
['/content/dataset/yolov8n.pt']


In [ ]:
# YOLO Configuration Setup
# =======================
# Configure the YOLO dataset configuration file for Google Colab environment
# This step updates the dataset paths to work correctly in the Colab runtime

import yaml, pathlib

# Load the original YOLO configuration file
src = "/content/dataset/plates_yolo/yolo.yaml"
with open(src) as f:
    cfg = yaml.safe_load(f)

# Update the dataset path to absolute path for Colab compatibility
cfg["path"] = "/content/dataset/plates_yolo"  # Set absolute path to dataset

# Save the updated configuration as a new file
fixed = "/content/dataset/plates_yolo/yolo_colab.yaml"
pathlib.Path(fixed).write_text(yaml.safe_dump(cfg))
print("Wrote:", fixed)

Wrote: /content/dataset/plates_yolo/yolo_colab.yaml


In [ ]:
# YOLOv8 Model Training
# ====================
# This is the main training cell that fine-tunes the YOLOv8 model for license plate detection
# The training process uses transfer learning from pre-trained weights

from ultralytics import YOLO

# Load the pre-trained YOLOv8 nano model
model = YOLO("/content/dataset/yolov8n.pt")

# Start training with optimized hyperparameters
results = model.train(
    # Dataset configuration
    data="/content/dataset/plates_yolo/yolo_colab.yaml",  # Path to dataset config
    
    # Training parameters
    epochs=50,        # Number of training epochs
    imgsz=640,        # Input image size (YOLO standard)
    device=0,         # Use GPU 0 (Tesla T4 in Colab)
    batch=16,         # Batch size for training
    workers=2,        # Number of data loading workers
    
    # Optimization settings
    amp=True,         # Automatic Mixed Precision for faster training
    optimizer="AdamW", # AdamW optimizer for better convergence
    patience=20,      # Early stopping patience
    
    # Output settings
    name="plate_yolov8n_colab",  # Experiment name
    project="/content/runs",      # Project directory
    plots=False,      # Disable plots to prevent font download issues
)

# Print the directory where training results are saved
print(results.save_dir)

Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/plates_yolo/yolo_colab.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/dataset/yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=plate_yolov8n_colab4, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, persp

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 92.0±26.5 MB/s, size: 545.4 KB)


train: Scanning /content/dataset/plates_yolo/labels/train... 347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 347/347 [00:01<00:00, 297.24it/s]

train: New cache created: /content/dataset/plates_yolo/labels/train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 97.2±25.4 MB/s, size: 571.8 KB)


...

      50/50      2.42G      1.164     0.6543      1.206         12        640: 100%|██████████| 22/22 [00:07<00:00,  2.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.03it/s]

                   all         86         96      0.884      0.865      0.902      0.486



50 epochs completed in 0.133 hours.
Optimizer stripped from /content/runs/plate_yolov8n_colab4/weights/last.pt, 6.2MB
Optimizer stripped from /content/runs/plate_yolov8n_colab4/weights/best.pt, 6.2MB

Validating /content/runs/plate_yolov8n_colab4/weights/best.pt...
Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.91it/s]


                   all         86         96      0.878      0.854      0.895      0.516
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 1.8ms postprocess per image
/content/runs/plate_yolov8n_colab4


In [ ]:
# Save Trained Model to Google Drive
# =================================
# This cell saves the best trained model weights to Google Drive for permanent storage
# This ensures the model is preserved even after the Colab session ends

from google.colab import drive
import glob

# Mount Google Drive to access persistent storage
drive.mount('/content/drive')

# Find the most recent best model weights from training runs
# The glob pattern matches all training runs and sorts by name to get the latest
best = sorted(glob.glob("/content/runs/plate_yolov8n_colab*/weights/best.pt"))[-1]

# Create output directory in Google Drive
!mkdir -p "/content/drive/MyDrive/yolo_outputs"

# Copy the best model weights to Google Drive
!cp -f "$best" "/content/drive/MyDrive/yolo_outputs/best.pt"
print("Saved to /content/drive/MyDrive/yolo_outputs/best.pt")

Mounted at /content/drive
Saved to /content/drive/MyDrive/yolo_outputs/best.pt


In [ ]:
# Dataset Structure Overview
# =========================
# Display the complete directory structure of the uploaded dataset
# This helps verify that all files are properly organized and accessible

!ls -R /content/dataset